# Cleaning CORPWATCH pre-processed file

This notebook reads the pre-processed CORPWATCH dataset from local data folder and performs the cleaning of country and company's names. The final result is saved locally and in the CORPWATCH S3 bucket.

In [1]:
import pandas as pd
import os

In [2]:
# Import CountryCleaner
from financial_entity_cleaner.location import CountryCleaner

In [3]:
# Import the module for cleaning company's name
from financial_entity_cleaner.company import CompanyNameCleaner

In [14]:
# Import BankingIdCleaner
from financial_entity_cleaner.id import BankingIdCleaner

## 1. Read CORPWATCH pre-processed file

In [4]:
# Save locally to "data" folder
saved_path = "..\..\data\pre_processed"
filename = "corpwatch_pre_processed.csv"

In [6]:
corpwatch_df = pd.read_csv(os.path.join(saved_path, filename), low_memory=False)

In [7]:
corpwatch_df.head()

,cw_id,cik,company_name,city,country_code,company_name_lei,lei
0,781188.0,13.0,CORPORATE INCOME FUND SEVENTY NINTH SHORT TERM...,NEW YORK,US,NaN,NaN
1,1.0,20.0,K TRON INTERNATIONAL INC,PITMAN,US,NaN,NaN
2,1249859.0,63.0,FNW BANCORP INC,ELGIN,US,NaN,NaN
3,2.0,1750.0,AAR CORP,WOOD DALE,US,Aar Corp,MP76T5YQX3YK5VVAQ802
4,3.0,1800.0,ABBOTT LABORATORIES,ABBOTT PARK,US,Abbott Laboratories,549300ZYY7524VF6JP88


## 2. Cleaning County information

In [8]:
# Create an object based on CountryCleaner() class
country_cleaner_obj=CountryCleaner()

In [9]:
country_cleaner_obj.letter_case='lower'
country_cleaner_obj.output_info = [CountryCleaner.ATTRIBUTE_ALPHA2,
                                   CountryCleaner.ATTRIBUTE_SHORT_NAME]

In [10]:
corpwatch_df = country_cleaner_obj.clean_df(df=corpwatch_df, cols=['country_code'], output_names_as= 'suffix')

Cleaning column [country_code]: 100%|██████████████████████████████████████████████████| 719598/719598 [38:47<00:00, 309.23it/s]


In [12]:
corpwatch_df.drop('country_code', axis=1, inplace=True)

In [13]:
corpwatch_df.head()

,cw_id,cik,company_name,city,company_name_lei,lei,country_code_short_name,country_code_alpha2
0,781188.0,13.0,CORPORATE INCOME FUND SEVENTY NINTH SHORT TERM...,NEW YORK,NaN,NaN,united states,us
1,1.0,20.0,K TRON INTERNATIONAL INC,PITMAN,NaN,NaN,united states,us
2,1249859.0,63.0,FNW BANCORP INC,ELGIN,NaN,NaN,united states,us
3,2.0,1750.0,AAR CORP,WOOD DALE,Aar Corp,MP76T5YQX3YK5VVAQ802,united states,us
4,3.0,1800.0,ABBOTT LABORATORIES,ABBOTT PARK,Abbott Laboratories,549300ZYY7524VF6JP88,united states,us


## 3. Cleaning LEI

In [15]:
# Create an object based on the BankingIdCleaner() class
id_cleaner_obj = BankingIdCleaner()

In [17]:
# Setup cleaning properties
id_cleaner_obj.invalid_ids_as_nan = True
id_cleaner_obj.validation_as_categorical = True
id_cleaner_obj.output_lettercase = "BankingIdCleaner.UPPER_LETTER_CASE"

In [18]:
corpwatch_df = id_cleaner_obj.clean_df(corpwatch_df, cols=['lei'], 
                                     remove_cols= True, 
                                     output_names_as= 'suffix',
                                     types = ['lei'])

Column [lei] Type [lei] : 100%|██████████████████████████████████████████████████| 719598/719598 [38:54<00:00, 308.19it/s]


In [21]:
corpwatch_df.drop('lei_isvalid_id', axis=1, inplace=True)

In [22]:
corpwatch_df.head()

,cw_id,cik,company_name,city,company_name_lei,country_code_short_name,country_code_alpha2,lei_cleaned_id
0,781188.0,13.0,CORPORATE INCOME FUND SEVENTY NINTH SHORT TERM...,NEW YORK,NaN,united states,us,NaN
1,1.0,20.0,K TRON INTERNATIONAL INC,PITMAN,NaN,united states,us,NaN
2,1249859.0,63.0,FNW BANCORP INC,ELGIN,NaN,united states,us,NaN
3,2.0,1750.0,AAR CORP,WOOD DALE,Aar Corp,united states,us,MP76T5YQX3YK5VVAQ802
4,3.0,1800.0,ABBOTT LABORATORIES,ABBOTT PARK,Abbott Laboratories,united states,us,549300ZYY7524VF6JP88


## 4. Cleaning Company's names

In [23]:
# Create a CompanyNameCleaner object
company_cleaner_obj = CompanyNameCleaner()

In [24]:
# Set cleaner properties
company_cleaner_obj.normalize_legal_terms = True
company_cleaner_obj.letter_case="lower"
company_cleaner_obj.remove_accents = True
company_cleaner_obj.default_cleaning_rules = ['place_word_the_at_the_beginning',
                                              'remove_words_in_asterisk', 
                                              'remove_words_in_parentheses',
                                              'remove_question_marks_in_parentheses', 
                                              'replace_hyphen_by_space', 
                                              'replace_underscore_by_space', 
                                              'remove_text_puctuation_except_dot', 
                                              'remove_math_symbols', 
                                              'remove_parentheses', 
                                              'remove_brackets', 
                                              'remove_curly_brackets', 
                                              'remove_single_quote_next_character', 
                                              'remove_double_quote', 
                                              'enforce_single_space_between_words']

In [25]:
corpwatch_df = company_cleaner_obj.clean_df(corpwatch_df, 'company_name', 'company_name_clean', 'country_code_alpha2', 'True')

In [26]:
corpwatch_df = company_cleaner_obj.clean_df(corpwatch_df, 'company_name_lei', 'company_name_lei_clean', 'country_code_alpha2', 'True')

In [42]:
corpwatch_df.reset_index(inplace=True)
corpwatch_df = corpwatch_df.rename(columns = {'index':'company_id'})

In [45]:
corpwatch_df.head()

,company_id,cw_id,cik,company_name,city,company_name_lei,country_code_short_name,country_code_alpha2,lei_cleaned_id,company_name_clean,company_name_lei_clean
0,0,781188.0,13.0,CORPORATE INCOME FUND SEVENTY NINTH SHORT TERM...,NEW YORK,NaN,united states,us,NaN,corporate income fund seventy ninth short term...,NaN
1,1,1.0,20.0,K TRON INTERNATIONAL INC,PITMAN,NaN,united states,us,NaN,k tron international incorporated,NaN
2,2,1249859.0,63.0,FNW BANCORP INC,ELGIN,NaN,united states,us,NaN,fnw bancorp incorporated,NaN
3,3,2.0,1750.0,AAR CORP,WOOD DALE,Aar Corp,united states,us,MP76T5YQX3YK5VVAQ802,aar corporation,aar corporation
4,4,3.0,1800.0,ABBOTT LABORATORIES,ABBOTT PARK,Abbott Laboratories,united states,us,549300ZYY7524VF6JP88,abbott laboratories,abbott laboratories


In [46]:
print('Total companies in CORPWATCH {}'.format(corpwatch_df.shape[0]))

Total companies in CORPWATCH 719598


## 4. Save cleaned CORPWATCH

In [47]:
# Save locally to "data" folder
saved_path = "..\..\data\cleaned"
filename = "corpwatch_cleaned.csv"
corpwatch_filename = os.path.join(saved_path, filename)
corpwatch_df.to_csv(corpwatch_filename, header=True, index=False)

In [32]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [33]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket_name = os.environ['S3_LANDING_BUCKET']
bucket = s3_resource.Bucket(bucket_name)

In [34]:
# Save final results to S3
s3_filename = 'CorpWatch/cleaned/corpwatch_cleaned.csv'
s3_resource.meta.client.upload_file(Filename=corpwatch_filename, Bucket=bucket_name, Key=s3_filename)